In [18]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
from torchvision import models
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split

In [19]:
DATA_DIR = r"D:\coding\SKRIPSI\CNNLSTM\images"
BATCH_SIZE = 1
MAX_FRAMES = 200
IMG_SIZE = 512
EPOCHS = 20
LEARNING_RATE = 1e-3
DEVICE = "cuda"

## Mapping (labels)

In [20]:
id_to_label = {
    "Non-Sianotik": [1001, 1002, 1007, 1011, 1014, 1018, 1019, 1020, 1025, 1029, 1033, 1035, 1036, 1041, 1047, 1061, 1070, 1079, 1103, 1132],
    "Sianotik": [1010, 1012, 1015, 1028, 1037, 1050, 1064, 1074, 1085, 1092, 1099, 1111, 1113, 1120, 1129, 1145, 1146, 1147],
    "Normal": [1003, 1005, 1032, 1051, 1062, 1063, 1066, 1067, 1072, 1078, 1080, 1083, 1101, 1116, 1117, 1127, 1128, 1143, 1144],
}
label_to_idx = {label: i for i, label in enumerate(id_to_label)}
patient_to_label = {}
for label, ids in id_to_label.items():
    for pid in ids:
        patient_to_label[str(pid)] = label

In [ ]:
class HeartSequenceDataset(Dataset):
    def __init__(self, image_dir, transform=None, max_frames=MAX_FRAMES):
        self.image_dir = image_dir
        self.transform = transform
        self.max_frames = max_frames
        self.patient_dict = {}

        all_image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
        for path in sorted(all_image_paths):
            basename = os.path.basename(path)
            try:
                pid = basename.split("_")[1]
            except:
                continue
            if pid not in patient_to_label:
                continue
            if pid not in self.patient_dict:
                self.patient_dict[pid] = []
            self.patient_dict[pid].append(path)
            self.image_paths.append(path)  # <== Simpan semua path valid

        self.patient_ids = list(self.patient_dict.keys())
        print(f"Total valid patients found: {len(self.patient_ids)}")

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, idx):
        pid = self.patient_ids[idx]
        image_paths = sorted(self.patient_dict[pid])[:self.max_frames]

        frames = []
        for img_path in image_paths:
            img = Image.open(img_path).convert("L").resize((IMG_SIZE, IMG_SIZE))
            img_tensor = transforms.ToTensor()(img)
            frames.append(img_tensor)

        # Padding if frames < max
        while len(frames) < self.max_frames:
            frames.append(torch.zeros_like(frames[0]))

        video_tensor = torch.stack(frames)  # shape: (T, C, H, W)
        label = label_to_idx[patient_to_label[pid]]

        return video_tensor, label


## Model

### CNN LSTM

In [22]:
class CNNLSTM(nn.Module):
    def __init__(self, hidden_dim=128, num_classes=3):
        super(CNNLSTM, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((8, 8))
        )
        self.lstm = nn.LSTM(input_size=16*8*8, hidden_size=hidden_dim, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: (B, T, C, H, W)
        B, T, C, H, W = x.size()
        x = x.view(B * T, C, H, W)
        features = self.cnn(x)  # (B*T, C, H, W)
        features = features.view(B, T, -1)
        lstm_out, _ = self.lstm(features)
        last_hidden = lstm_out[:, -1, :]
        output = self.classifier(last_hidden)
        return output

## Training

In [23]:
def get_dataset_splits(dataset, test_size=0.2):
    labels = [label_to_idx[patient_to_label[pid]] for pid in dataset.patient_ids]
    train_idx, val_idx = train_test_split(range(len(labels)), test_size=test_size, stratify=labels, random_state=42)
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)
    return train_dataset, val_dataset, [labels[i] for i in train_idx]

In [ ]:
def train_model():
    dataset = HeartSequenceDataset(DATA_DIR)
    train_dataset, val_dataset, train_labels = get_dataset_splits(dataset)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=0)

    model = CNNLSTM().to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    label_counts = Counter(train_labels)
    total = sum(label_counts.values())
    class_weights = [total / label_counts[i] for i in range(3)]
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).float().to(DEVICE))

    for epoch in range(EPOCHS):
        model.train()
        total_loss, correct = 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        acc = correct / len(train_loader.dataset)
        print(f"Train Loss: {total_loss:.4f} | Train Acc: {acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                val_correct += (outputs.argmax(1) == labels).sum().item()
        val_acc = val_correct / len(val_loader.dataset)
        print(f"Val Acc: {val_acc:.4f}\n")

    torch.save(model.state_dict(), "cnn_lstm_model.pth")


In [25]:
if __name__ == "__main__":
    train_model()

Total valid patients found: 57


Epoch 1/20:   0%|          | 0/45 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7780\1050181036.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/20: 100%|██████████| 45/45 [02:03<00:00,  2.74s/it]


Train Loss: 50.4072 | Train Acc: 0.2889
Val Acc: 0.3333



Epoch 2/20: 100%|██████████| 45/45 [01:47<00:00,  2.38s/it]


Train Loss: 49.5688 | Train Acc: 0.3556
Val Acc: 0.3333



...

Train Loss: 40.2218 | Train Acc: 0.5778
Val Acc: 0.2500



Epoch 19/20: 100%|██████████| 45/45 [01:43<00:00,  2.31s/it]


Train Loss: 34.6385 | Train Acc: 0.6889
Val Acc: 0.4167



Epoch 20/20: 100%|██████████| 45/45 [01:42<00:00,  2.29s/it]


Train Loss: 33.1981 | Train Acc: 0.6444
Val Acc: 0.3333



## Evaluation

In [26]:
from sklearn.metrics import classification_report

def evaluate_model(model, dataloader, class_names=['Non-Sianotik', 'Sianotik', 'Normal']):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))


In [28]:
# Load trained model
model = CNNLSTM().to(DEVICE)
model.load_state_dict(torch.load("cnn_lstm_model.pth"))

# Load validation dataset
dataset = HeartSequenceDataset(DATA_DIR)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
_, val_dataset = random_split(dataset, [train_size, val_size])
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=0)

# Evaluate model
evaluate_model(model, val_loader)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7780\2020444447.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("cnn_lstm_model.pth"))


Total valid patients found: 57

Classification Report:
              precision    recall  f1-score   support

Non-Sianotik       1.00      0.80      0.89         5
    Sianotik       0.67      1.00      0.80         2
      Normal       1.00      1.00      1.00         5

    accuracy                           0.92        12
   macro avg       0.89      0.93      0.90        12
weighted avg       0.94      0.92      0.92        12

